In [1]:
### polygonio_data_puller
#Forest Kirschbaum 2022_10_26

###most inspirational
#Abhishek Gupta
#https://greyhoundanalytics.com/blog/scraping-crypto-price-data-from-polygonio/


# https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to
# https://raw.githubusercontent.com/AdamGetbags/polygonData/main/historicalOptionDataPolygon.py
# https://github.com/pssolanki111/polygon
# https://polygon.readthedocs.io/en/latest/bulk_data_download_functions.html
# https://github.com/AdamGetbags/polygonData/blob/main/historicalOptionDataPolygon.py
# https://www.geeksforgeeks.org/python-pandas-series-tz_convert/
# https://www.geeksforgeeks.org/python-pandas-series-tz_convert/



#     polygon_api_key = "fake_key_enter_yours_here" #get your own free from polygon.io
polygon_api_key = "fake_key_enter_yours_here"

In [2]:

import requests
import pandas as pd

from datetime import date, datetime, timedelta
import time
import os
from polygon import RESTClient
import datetime as dt

In [3]:
"""
start of user entered variables
"""


'\nstart of user entered variables\n'

In [4]:
##asset_type = "stock" or "crypto"      ##stock like: AAPL, TSLA # crypto like: BTCUSD, UNIUSD, ETHUSD
asset_type = "stock"
asset = "AAPL"


In [5]:
lookback_months = 25

In [6]:
# 2022_10_17_jupyter = <<your python project in jupyter notebook or selected editor>>
# data/polygon_assets = <<the path starting in dir, then adding onto where you want to store your data csvs.
DIR = r'/home/guest144/backtrader-master/'
target_dir = os.path.join(DIR, 'datas/polygonio')

In [7]:
"""
end of user entered variables
"""

'\nend of user entered variables\n'

In [8]:
if asset_type in ['stock']:
    asset = asset
elif asset_type in ['crypto']:
    asset = "X:" + asset

In [9]:
def pull_1m_data(asset, start_time, end_time):

    polygon_rest_baseurl = "https://api.polygon.io/v2/"
    multiplier = 1
    timespan = "minute"
    limit = 50000
    # newest data at the bottom
    sort = "asc"
        
    request_url = f"{polygon_rest_baseurl}aggs/ticker/{asset}/range/{multiplier}/" +\
            f"{timespan}/{start_time}/{end_time}?adjusted=true&sort={sort}&" + \
            f"limit={limit}&apiKey={polygon_api_key}"

    data = requests.get(request_url).json()

    if "results" in data:
        return data["results"]
    else:
        raise Exception("error_on_pull_1m_data")

In [10]:
bars = []

for i in range(lookback_months):
    end_time = date.today() - i*timedelta(days = 28)
    start_time = date.today() - (i+1)*timedelta(days = 28)
    bars += pull_1m_data(asset, start_time, end_time)
    time.sleep(15)

In [20]:
df = pd.DataFrame(bars)
df["date"] = pd.to_datetime(df["t"], unit = "ms")
df =  df[["date","o","h","l","c","v"]]
df =df.rename(columns = {
    "date":"time",
    "o": "Open",
    "h": "High",
    "l": "Low",
    "c": "Close",
    "v": "Volume"})
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
#localize time replace US/Central with <<your timezone>>
df.index = df.index.tz_localize('UTC')
df.index = df.index.tz_convert('US/Central') # df.index = df.index.tz_convert('America/New_York')
df = df.sort_values("time")

datapath = os.path.join(target_dir, asset + '.csv')
df.to_csv(datapath)

print(df)

                             Open    High     Low   Close   Volume
time                                                              
2020-12-01 03:00:00-06:00  120.84  120.87  119.95  120.64  11000.0
2020-12-01 03:01:00-06:00  120.59  120.78  120.59  120.73   1453.0
2020-12-01 03:02:00-06:00  120.70  120.79  120.70  120.73   2452.0
2020-12-01 03:03:00-06:00  120.75  120.85  120.75  120.85   2389.0
2020-12-01 03:04:00-06:00  120.91  120.91  120.88  120.88   1328.0
...                           ...     ...     ...     ...      ...
2022-10-31 18:53:00-05:00  153.45  153.45  153.45  153.45    570.0
2022-10-31 18:55:00-05:00  153.45  153.46  153.43  153.46   1236.0
2022-10-31 18:56:00-05:00  153.46  153.46  153.45  153.46    955.0
2022-10-31 18:58:00-05:00  153.47  153.47  153.45  153.46   1032.0
2022-10-31 18:59:00-05:00  153.46  153.50  153.46  153.46   1691.0

[416597 rows x 5 columns]
